In [8]:
import os
import numpy as np
import sys
import pandas as pd
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.utils.np_utils import to_categorical

In [5]:
data = pd.read_csv('/home/paul/Documents/AI/Datasets/fer2013.csv')

#print(data.info())
#print(data['Usage'].value_counts())
#print(data.head())

x_train, y_train, x_test, y_test = [], [], [], []

for index, row in data.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            x_train.append(np.array(val, 'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            x_test.append(np.array(val, 'float32'))
            y_test.append(row['emotion'])
    except:
        print(f'error occured at index:{index} and row:{row}')

num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48
              
print(f'x_train sample data{x_train[0:2]}')
print(f'y_train sample data{x_test[0:2]}')
print(f'x_test sample data{y_train[0:2]}')
print(f'y_test sample data{x_test[0:2]}')

x_train = np.array(x_train, 'float32')
x_test = np.array(x_test, 'float32')
y_train = np.array(y_train, 'float32')
y_test = np.array(y_test, 'float32')

y_train=to_categorical(y_train, num_classes=num_labels)
y_test=to_categorical(y_test, num_classes=num_labels)

#Normalizing Data
x_train -= np.mean(x_train, axis=0)
x_train /= np.mean(x_train, axis=0)

x_test -= np.mean(x_test, axis=0)
x_test /= np.mean(x_test, axis=0)

x_train = x_train.reshape(x_train.shape[0], width, height, 1)
x_test = x_test.reshape(x_test.shape[0], width, height, 1)

x_train sample data[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
y_train sample data[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
x_test sample data[0, 0]
y_test sample data[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]


In [6]:
model = Sequential()

#first convolution layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(x_train.shape[1:])))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

#second convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

#third convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(1204, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1204, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            shuffle=True)

fer_json=model.to_json()
with open('fer_model.json','w') as json_file:
    json_file.write(fer_json)
model.save_weights('fer_weights.h5')

Epoch 1/50
449/449 [==============================] - 246s 548ms/step - loss: 1.8476 - accuracy: 0.2434 - val_loss: 2.3794 - val_accuracy: 0.1792
Epoch 2/50
449/449 [==============================] - 249s 555ms/step - loss: 1.7894 - accuracy: 0.2554 - val_loss: 2.1601 - val_accuracy: 0.1953
Epoch 3/50
449/449 [==============================] - 249s 556ms/step - loss: 1.7572 - accuracy: 0.2765 - val_loss: 2.0374 - val_accuracy: 0.1688
Epoch 4/50
449/449 [==============================] - 249s 556ms/step - loss: 1.7230 - accuracy: 0.3061 - val_loss: 1.9694 - val_accuracy: 0.1775
Epoch 5/50
449/449 [==============================] - 250s 556ms/step - loss: 1.6910 - accuracy: 0.3259 - val_loss: 2.4838 - val_accuracy: 0.2048
Epoch 6/50
449/449 [==============================] - 250s 556ms/step - loss: 1.6651 - accuracy: 0.3375 - val_loss: 2.2558 - val_accuracy: 0.1864
Epoch 7/50
449/449 [==============================] - 249s 554ms/step - loss: 1.6400 - accuracy: 0.3534 - val_loss: 2.4045 -

In [7]:
with open('femodel.json','w') as json_file:
    json_file.write(fer_json)
model.save_weights('feweights.h5')

In [13]:
data = pd.read_csv('/home/paul/Documents/AI/Datasets/fer2013.csv')

#print(data.info())
#print(data['Usage'].value_counts())
#print(data.head())
#print(data)

emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]
